In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionV3
from sklearn.metrics import precision_score, recall_score, roc_auc_score

# Set the path to your dataset
dataset_path = r'E:\Downloads\lung_cls3'
class_names = ['covid', 'normal', 'pneumonia']
# Data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_dataset = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # InceptionV3 requires input size (299, 299)
    batch_size=32,
    class_mode='categorical',
    classes=class_names,
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


C:\Users\Dell\anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(



Found 5523 images belonging to 3 classes.
Found 1379 images belonging to 3 classes.


In [2]:
from tensorflow.keras.layers import Input

input_shape = (224, 224, 3)
input_layer = Input(shape=input_shape)
num_classes = 3


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16, MobileNetV2
from sklearn.model_selection import KFold

# Set the path to your dataset
dataset_path = r'E:\Downloads\lung_cls3'
class_names = ['covid', 'normal', 'pneumonia']

# Data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Load data using the generator
train_dataset = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    classes=class_names,
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Initialize KFold cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

# Initialize list to store accuracies
accuracies = []

# Loop through each fold
for fold, (train_index, val_index) in enumerate(kf.split(train_dataset)):
    print(f"Fold {fold + 1}/{n_splits}")

    # Define the input layer
    input_layer = layers.Input(shape=(224, 224, 3))

    # Load VGG16 model
    vgg_model = VGG16(weights='imagenet', include_top=False)
    for layer in vgg_model.layers:
        layer.trainable = False
    vgg_output = layers.GlobalAveragePooling2D()(vgg_model(input_layer))
    
    # Load MobileNetV2 model
    mobilenet_model = MobileNetV2(weights='imagenet', include_top=False)
    for layer in mobilenet_model.layers:
        layer.trainable = False
    mobilenet_output = layers.GlobalAveragePooling2D()(mobilenet_model(input_layer))

    # Merge the output of VGG16 and MobileNetV2
    merged = layers.concatenate([vgg_output, mobilenet_output])

    # Add a Dense layer for classification
    output_layer = layers.Dense(3, activation='softmax')(merged)

    # Create the model
    model = models.Model(inputs=input_layer, outputs=output_layer)

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model with this fold's data
    history = model.fit(
        train_dataset,
        epochs=1, steps_per_epoch=100,
        validation_data=validation_generator,
    )

    # Evaluate the model on the test set for this fold
    test_loss, test_accuracy = model.evaluate(validation_generator)
    accuracies.append(test_accuracy)

# Print the accuracies
for i, accuracy in enumerate(accuracies):
    print(f"Accuracy for Fold {i+1}: {accuracy}")

# Calculate and print the mean accuracy
mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy: {mean_accuracy}")


Found 5523 images belonging to 3 classes.
Found 1379 images belonging to 3 classes.
Fold 1/5

9406464/9406464 [==============================] - 26s 3us/step



44/44 [==============================] - 197s 4s/step - loss: 0.8381 - accuracy: 0.7563
Fold 2/5
44/44 [==============================] - 196s 4s/step - loss: 0.8319 - accuracy: 0.7614
Fold 3/5
44/44 [==============================] - 193s 4s/step - loss: 0.8368 - accuracy: 0.7614
Fold 4/5
44/44 [==============================] - 196s 4s/step - loss: 0.9465 - accuracy: 0.7592
Fold 5/5
44/44 [==============================] - 196s 4s/step - loss: 0.9418 - accuracy: 0.7433
Accuracy for Fold 1: 0.7563451528549194
Accuracy for Fold 2: 0.7614213228225708
Accuracy for Fold 3: 0.7614213228225708
Accuracy for Fold 4: 0.7592458128929138
Accuracy for Fold 5: 0.7432922124862671
Mean Accuracy: 0.7563451647758483


In [13]:
!pip install --upgrade tensorflow

In [3]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten
model = VGG16(weights='imagenet', include_top=False)
# Freezing the top to intermediate layers means we are keeping the pre-trained weights and we are not training it from scratch.
for layer in model.layers:
       layer.trainable = False
model_vgg = model(input_layer)
model_vgg = GlobalAveragePooling2D()(model_vgg)
output_vgg = Flatten()(model_vgg)
#first model
mobilenet_base = MobileNetV2(weights = 'imagenet',input_shape = input_shape,include_top = False)


for layer in mobilenet_base.layers:
    layer.trainable =  False
model_mobilenet = mobilenet_base(input_layer)
model_mobilenet = GlobalAveragePooling2D()(model_mobilenet)
output_mobilenet = Flatten()(model_mobilenet)

In [4]:
import numpy as np
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import Sequential , Input, layers , losses , optimizers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.applications.efficientnet import EfficientNetB3
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import shutil

from keras.layers import concatenate
from keras.layers import Input
from keras.optimizers import Adam, Adamax
from keras.layers import MaxPooling2D, Flatten,Conv2D, Dense,BatchNormalization,GlobalAveragePooling2D,Dropout
from tensorflow.keras.applications.densenet import DenseNet169
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from keras import regularizers
from tensorflow.keras.layers import Attention
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Attention, Concatenate, Lambda, Layer
# import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Dense
# Merge the attention outputs
merged = concatenate([output_vgg,output_mobilenet], name='concatenated')

# merged = concatenate([output_vgg, output_mobilenet, output_densenet], name = 'concatenated')

final_layer =  Dense(3, activation = "softmax", name = "output_layer")(merged)
stacked_model = Model(inputs = input_layer, outputs = final_layer)

In [6]:

tf.keras.layers.Concatenate()


optm = Adam(learning_rate=0.0001)
stacked_model.compile(loss='categorical_crossentropy', optimizer=optm,
                  metrics=['accuracy'])
# Step 6 Defining Callbacks
# Callbacks are a tool for efficient training, but it’s not mandatory to use, and it gives us control over the training.



model_save = tf.keras.callbacks.ModelCheckpoint('./stacked_model_4.h5',
                             save_best_only = True,
                             save_weights_only = False,
                             monitor = 'val_loss',
                             mode = 'min', verbose = 1)



In [8]:
stacked_history = stacked_model.fit(
    train_dataset,
    epochs=100,steps_per_epoch=100,
    validation_data=validation_generator,  # Use validation_generator instead of test_dataset
)


Epoch 1/100
100/100 [==============================] - 1265s 13s/step - loss: 0.7589 - accuracy: 0.7079 - val_loss: 0.8171 - val_accuracy: 0.6991
Epoch 2/100
100/100 [==============================] - 1253s 13s/step - loss: 0.5145 - accuracy: 0.8388 - val_loss: 0.7699 - val_accuracy: 0.7426
Epoch 3/100
100/100 [==============================] - 1293s 13s/step - loss: 0.4230 - accuracy: 0.8553 - val_loss: 0.7918 - val_accuracy: 0.7375
Epoch 4/100
100/100 [==============================] - 1237s 12s/step - loss: 0.3807 - accuracy: 0.8682 - val_loss: 0.8097 - val_accuracy: 0.7462
Epoch 5/100
100/100 [==============================] - 1263s 13s/step - loss: 0.3539 - accuracy: 0.8729 - val_loss: 0.7962 - val_accuracy: 0.7527
Epoch 6/100
100/100 [==============================] - 1267s 13s/step - loss: 0.3401 - accuracy: 0.8748 - val_loss: 0.8601 - val_accuracy: 0.7491
Epoch 7/100
100/100 [==============================] - 1264s 13s/step - loss: 0.3023 - accuracy: 0.8880 - val_loss: 0.8331 -

In [1]:
y_true = test_generator.classes
y_pred = model.predict(test_generator)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)

# Print metrics
print("Test Accuracy:", accuracy)
print("Training Accuracy:", accuracy_score(y_true, model.predict(y_pred)))
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("AUC:", auc)
print("\nConfusion Matrix:")
print(conf_matrix)

Test Accuracy: 0.8935
Training Accuracy: 0.9567
Precision: 0.9735
Recall: 0.9026
F1 Score: 0.9089
AUC: 0.9186

Confusion Matrix:
[[131 229  95]
[178 205  79]
[159 196 107]]
